# Importing data

In [1]:
from scipy.stats import chi2_contingency as chi_test
import pandas as pd
import numpy as np
import pycm

# Importing data

In [2]:
data = pd.read_excel(r"D:\Academics\CMI\Internship\Fortiate\DST-1_Dimensionality\Data\DATA FOR ALL TEAMS-25MB.xlsx")

In [3]:
data.head(2)

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,...,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
0,DE2,DE3,DE4,DE5,DE6,DE7,DE9,DE14,DE16,DE18,...,DE4801,DE4801,DE4803,DE4804,DE4805,DE4806,DE4807,DE4808,DE4809,DE4810
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,...,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN


# Preprocessing

### Header to code mapper

In [4]:
map_head_code = dict(list(zip(list(data.loc[0, :]), list(data.columns))))

### Removing 1st column, Code

In [5]:
data = data.drop(0)
# data.reset_index()

### Removing not required colums

In [6]:
not_required = ["CONVERSION_RATE_DATE", "Private Data 48", "Transaction Category Code", "DE48 Data Length", "Additional Payment Info", "Ecommerce Data", "ECI Indicator", "Private Data - Additional Info", "Token Data", "TARGET", "POS entry Mode"]

In [7]:
data = data.drop(not_required, axis=1)

In [8]:
data.head(2)

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,Merchant Category Code,ACQUIRING_COUNTRY_CODE,...,MID,MERCHANT NAME AND ADDRESS,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Remaining Balance
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,6011,356,...,1005044921,LXFC BTNK LTD.KHURDT OR IN,356,356,356,210201210141,IN1,ATM,0917,35000
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,6011,356,...,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,356,356,356,210201210141,IN1,ATM,0796,44000


### Seperating numerical and catagorical data

In [9]:
numerical = ["TRANSACTION_AMOUNT", "SETTLEMENT_AMOUNT", "BILLING_AMOUNT", "CONVERSION_RATE",  "Remaining Balance"]

In [10]:
data_cat = data.drop(numerical, axis = 1)

In [15]:
data_cat.head(2)

,Card-Wallet-Token ID,PROCESSING_CODE,Transaction Date and Time,Expiry Date,Merchant Category Code,ACQUIRING_COUNTRY_CODE,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1
1,6086941819072006,174000,22-MAR-18 09.10.37,01-JUL-23 00.00.00,6011,356,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,356,356,356,210201210141,IN1,ATM,0917
2,6086941026163002,174000,22-MAR-18 12.28.15,01-DEC-22 00.00.00,6011,356,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,356,356,356,210201210141,IN1,ATM,0796


# Independence check

### Goodmans kruskal gamma

##### F1 score of each pair with null hypothesis they are independent

F1 score 1 implies complete dependencies and 0 implies independet

In [12]:
cat_columns = ['PROCESSING_CODE',
     'Merchant Category Code',
     'ACQUIRING_COUNTRY_CODE',
     'ACQUIRER_INSTITUTION_CODE',
     'Card Service Code',
     'Interface ID',
     'Terminal Type',
     'Token Data-C1']

In [55]:
# cat_columns = list(data_cat.columns)
count = 0
dependent_pair_goodman = []
for i in cat_columns:    
#     print("\n")
#     print(i)
    count += 1
    for j in cat_columns[count:]:
        y_actu = list(data_cat[i])
        y_pred = list(data_cat[j])
        cm = pycm.ConfusionMatrix(actual_vector=y_actu, predict_vector=y_pred)
#         print("\t",j, cm.F1_Macro)
        if cm.F1_Macro > 0.005:
            dependent_pair_goodman.append([i,j])

In [56]:
dependent_pair_goodman

[['PROCESSING_CODE', 'ACQUIRING_COUNTRY_CODE'],
 ['PROCESSING_CODE', 'Card Service Code'],
 ['PROCESSING_CODE', 'Interface ID'],
 ['PROCESSING_CODE', 'Terminal Type'],
 ['ACQUIRING_COUNTRY_CODE', 'Card Service Code'],
 ['ACQUIRING_COUNTRY_CODE', 'Interface ID'],
 ['ACQUIRING_COUNTRY_CODE', 'Terminal Type'],
 ['Card Service Code', 'Interface ID'],
 ['Card Service Code', 'Terminal Type'],
 ['Interface ID', 'Terminal Type']]

### Chi-square test

##### With null hypothesis H0: They are independent

Thus p value < 0.01 implies we are rejecting the null hypothesis and there is some dependency

In [ ]:
# count = 0
# for i in cat_columns:    
#     print("\n")
#     print(i)
#     count += 1
#     for j in cat_columns[count:]:
#         data_crosstab =  pd.crosstab(data_cat[i], data_cat[j], margins = False)
#         g, p, dof, expctd = chi_test(data_crosstab, lambda_="log-likelihood")
#         print("\t",j, p)

In [13]:
#cat_columns = list(data_cat.columns)
count = 0
dependent_pair = []
for i in cat_columns:
    count += 1
    for j in cat_columns[count:]:
        data_crosstab =  pd.crosstab(data_cat[i], data_cat[j], margins = False)
        g, p, dof, expctd = chi_test(data_crosstab, lambda_="log-likelihood")
        if p < 0.01:
            dependent_pair.append([i,j])

In [14]:
dependent_pair

[['PROCESSING_CODE', 'Merchant Category Code'],
 ['PROCESSING_CODE', 'ACQUIRING_COUNTRY_CODE'],
 ['PROCESSING_CODE', 'ACQUIRER_INSTITUTION_CODE'],
 ['PROCESSING_CODE', 'Card Service Code'],
 ['PROCESSING_CODE', 'Interface ID'],
 ['PROCESSING_CODE', 'Terminal Type'],
 ['Merchant Category Code', 'Card Service Code'],
 ['Merchant Category Code', 'Interface ID'],
 ['Merchant Category Code', 'Terminal Type'],
 ['ACQUIRING_COUNTRY_CODE', 'Card Service Code'],
 ['ACQUIRING_COUNTRY_CODE', 'Interface ID'],
 ['ACQUIRER_INSTITUTION_CODE', 'Card Service Code'],
 ['ACQUIRER_INSTITUTION_CODE', 'Interface ID'],
 ['ACQUIRER_INSTITUTION_CODE', 'Terminal Type'],
 ['Card Service Code', 'Interface ID'],
 ['Card Service Code', 'Terminal Type'],
 ['Card Service Code', 'Token Data-C1'],
 ['Interface ID', 'Terminal Type'],
 ['Interface ID', 'Token Data-C1']]

### Cramer's V test

In [42]:
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = chi_test(confusion_matrix)[0]
    #print(chi2)
    n = (confusion_matrix.sum()).sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [45]:
count = 0
dependent_pair_cramer = []
for i in cat_columns:
    count += 1
    for j in cat_columns[count:]:
        data_crosstab =  pd.crosstab(data_cat[i], data_cat[j], margins = False)
        p = cramers_corrected_stat(data_crosstab)
        #if p < 0.01:
        dependent_pair_cramer.append([i,j,p])

In [46]:
dependent_pair_cramer

[['PROCESSING_CODE', 'Merchant Category Code', 0.3946169414516618],
 ['PROCESSING_CODE', 'ACQUIRING_COUNTRY_CODE', 0.11715815911243221],
 ['PROCESSING_CODE', 'ACQUIRER_INSTITUTION_CODE', 0.7279088781128282],
 ['PROCESSING_CODE', 'Card Service Code', 0.5411999747245039],
 ['PROCESSING_CODE', 'Interface ID', 0.823597310108287],
 ['PROCESSING_CODE', 'Terminal Type', 0.8519071431301805],
 ['PROCESSING_CODE', 'Token Data-C1', 0.0],
 ['Merchant Category Code', 'ACQUIRING_COUNTRY_CODE', 0.30916815336342074],
 ['Merchant Category Code', 'ACQUIRER_INSTITUTION_CODE', 0.32702241853922454],
 ['Merchant Category Code', 'Card Service Code', 0.4127593586343751],
 ['Merchant Category Code', 'Interface ID', 0.3393648314903789],
 ['Merchant Category Code', 'Terminal Type', 0.3970058167481686],
 ['Merchant Category Code', 'Token Data-C1', 0.07764079645552503],
 ['ACQUIRING_COUNTRY_CODE', 'ACQUIRER_INSTITUTION_CODE', 0.926921347160859],
 ['ACQUIRING_COUNTRY_CODE', 'Card Service Code', 0.10474922032910816]

# Rough